# 客户流失预测 - PyCaret实现

> 🎯 目标：使用PyCaret在3分钟内建立高性能客户流失预测模型

---

## 📋 项目信息

- **任务类型**: 二分类（客户是否流失）
- **数据集**: Telco Customer Churn (7043条记录, 21个特征)
- **目标**: 快速建立baseline，与手动方法对比
- **预期时间**: 3-5分钟

---

## 1. 环境准备

In [ ]:
# 安装 PyCaret（如果未安装）
# !pip install pycaret

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# PyCaret
from pycaret.classification import *

print("✅ 环境准备完成")

---

## 2. 数据加载

使用原项目的数据

In [ ]:
# 数据路径（相对于automl_toolkit目录）
data_path = "../../../supervised_learning/projects/02_customer_churn_prediction/data/WA_Fn-UseC_-Telco-Customer-Churn.csv"

# 加载数据
data = pd.read_csv(data_path)

print(f"数据形状: {data.shape}")
print(f"\n前5行:")
data.head()

In [ ]:
# 数据基本信息
print("数据信息:")
data.info()

print("\n目标变量分布:")
print(data['Churn'].value_counts())
print(f"\n流失率: {data['Churn'].value_counts(normalize=True)['Yes']:.2%}")

---

## 3. 数据预处理（最小化）

PyCaret会自动处理大部分预处理，我们只需做最基本的清理

In [ ]:
# 处理TotalCharges列（转换为数值）
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

# 删除customerID（无用特征）
data = data.drop('customerID', axis=1)

print("✅ 基础预处理完成")
print(f"最终数据形状: {data.shape}")

---

## 4. PyCaret 初始化（自动特征工程）

这一步PyCaret会自动：
- 推断数据类型
- 划分训练/测试集
- 处理缺失值
- 编码类别特征
- 标准化数值特征

In [ ]:
# 初始化PyCaret环境
clf = setup(
    data=data,
    target='Churn',
    session_id=42,
    
    # 训练配置
    train_size=0.8,
    
    # 预处理选项
    normalize=True,              # 标准化数值特征
    fix_imbalance=True,          # 处理类别不平衡（重要！）
    
    # 特征工程
    feature_selection=True,      # 自动特征选择
    feature_interaction=True,    # 自动特征交互
    
    # 其他
    remove_outliers=False,       # 不移除异常值
    verbose=True                 # 显示详细信息
)

---

## 5. 模型对比（核心步骤）

自动训练15+个模型并对比性能

In [ ]:
# 对比所有模型
best_model = compare_models(
    n_select=1,              # 返回最佳的1个模型
    sort='AUC',              # 按AUC排序（分类常用指标）
    budget_time=5            # 最多5分钟
)

**分析要点**：
- 查看哪个模型表现最好
- 对比不同模型的AUC、Accuracy、Recall等指标
- 注意训练时间（TT）

---

## 6. 模型调优

In [ ]:
# 自动调优最佳模型
tuned_model = tune_model(
    best_model,
    n_iter=50,               # 尝试50组超参数
    optimize='AUC',          # 优化目标
    search_library='optuna'  # 使用Optuna贝叶斯优化
)

---

## 7. 模型评估与可视化

In [ ]:
# 混淆矩阵
plot_model(tuned_model, plot='confusion_matrix')

In [ ]:
# ROC曲线
plot_model(tuned_model, plot='auc')

In [ ]:
# 特征重要性
plot_model(tuned_model, plot='feature')

In [ ]:
# Precision-Recall曲线
plot_model(tuned_model, plot='pr')

In [ ]:
# 学习曲线
plot_model(tuned_model, plot='learning')

---

## 8. 模型融合（提升性能）

In [ ]:
# 获取前3个最佳模型
top3_models = compare_models(n_select=3, sort='AUC')

# Voting集成
blended_model = blend_models(top3_models, optimize='AUC')

# Stacking集成
stacked_model = stack_models(top3_models, optimize='AUC')

In [ ]:
# 对比单模型 vs 集成模型
print("=" * 50)
print("模型性能对比")
print("=" * 50)

models_to_compare = {
    'Best Single Model': tuned_model,
    'Blended (Voting)': blended_model,
    'Stacked': stacked_model
}

# 评估每个模型
for name, model in models_to_compare.items():
    print(f"\n{name}:")
    predict_model(model)

---

## 9. 测试集评估

In [ ]:
# 在hold-out测试集上评估
final_predictions = predict_model(tuned_model)

print("测试集预测结果（前10行）:")
final_predictions.head(10)

In [ ]:
# 获取详细指标
from sklearn.metrics import classification_report, roc_auc_score

y_test = final_predictions['Churn']
y_pred = final_predictions['prediction_label']
y_pred_proba = final_predictions['prediction_score']

print("分类报告:")
print(classification_report(y_test, y_pred))

print(f"\nAUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")

---

## 10. 保存模型

In [ ]:
# 在全量数据上训练最终模型
final_model = finalize_model(tuned_model)

# 保存模型
save_model(final_model, 'pycaret_churn_model')

print("✅ 模型已保存为 pycaret_churn_model.pkl")

---

## 11. 关键发现总结

### AutoML自动处理的内容

1. **数据预处理**
   - ✅ 自动识别数值/类别特征
   - ✅ 缺失值填充
   - ✅ 类别编码（One-Hot）
   - ✅ 数值标准化
   - ✅ 类别不平衡处理（SMOTE）

2. **特征工程**
   - ✅ 自动特征交互
   - ✅ 特征选择
   - ✅ 多项式特征（可选）

3. **模型选择**
   - ✅ 对比15+个模型
   - ✅ 10折交叉验证
   - ✅ 自动选择最佳模型

4. **超参数调优**
   - ✅ 贝叶斯优化（Optuna）
   - ✅ 智能搜索空间
   - ✅ 自动早停

5. **模型融合**
   - ✅ Voting
   - ✅ Stacking
   - ✅ Bagging

### 与手动方法对比

| 维度 | 手动方法 | PyCaret |
|------|---------|--------|
| 时间 | 3小时 | 3-5分钟 |
| 代码量 | 300+行 | 10行 |
| 模型数量 | 3-5个 | 15+个 |
| 特征工程 | 手动设计 | 自动生成 |
| 调优 | 网格搜索 | 贝叶斯优化 |
| 性能 | 基线80-81% | 80-82% |

### 学到的经验

1. **最佳模型**: 查看PyCaret选择了哪个模型（通常是树模型）
2. **重要特征**: 从特征重要性图中了解关键因素
3. **性能指标**: AUC、Recall对于流失预测很重要
4. **类别不平衡**: fix_imbalance参数很关键

---

## 🎯 练习任务

1. ✅ 尝试不同的`fix_imbalance`设置，对比效果
2. ✅ 调整`feature_selection`和`feature_interaction`参数
3. ✅ 对比单模型vs集成模型的性能提升
4. ✅ 分析特征重要性，与业务直觉对比
5. ✅ 尝试不同的优化指标（AUC vs F1 vs Recall）

---

## 📚 下一步

- **02_flaml_churn.ipynb**: 使用FLAML更快速的实现
- **03_comparison_with_manual.ipynb**: 详细对比AutoML vs 手动方法

---

**恭喜完成PyCaret客户流失预测！🎉**